## spell checker

In [2]:
import re
import pandas as pd

In [3]:
with open('big.txt','r') as fd:
    lines = fd.readlines()
    words = []
    for line in lines:
        words += re.findall('\w+',line.lower())
        
print(len(words))
vocab = list(set(words))
print(len(vocab))

1115585
32198


## word probabilites

In [18]:
word_probability = {}

for word in (vocab):
    word_probability[word] = float(words.count(word)/len(words))

KeyboardInterrupt: 

In [5]:
def split(word):  
    parts = []
    for i in range(len(word) + 1):
        parts += [(word[ : i], word[i : ])]
    return parts

In [6]:
def delete(word):
    
    output = []
    for l,r in split(word):
        output.append(l + r[1:])
    return output

delete('loave')

['oave', 'lave', 'love', 'loae', 'loav', 'loave']

In [7]:
def swap(word):
        
    output = []    
    for l,r in split(word):
        if (len(r) > 1):
            output.append(l + r[1] + r[0] + r[2:])
    return output
            
swap('lvoe')

['vloe', 'love', 'lveo']

In [8]:
def replace(word):
    
    characters = 'abcdefghijklmnopqrstuvwxyz'
    output = []    

    for l,r in split(word):
        for char in characters:
            output.append(l + char +  r[1:])
    return output

len(replace('lave'))

130

In [14]:
def inserts(word):

    characters = 'abcdefghijklmnopqrstuvwxyz'
    output = []

    for l,r in split(word):
        for char in characters:
            output.append(l + char + r)

    return output

len(inserts('lve'))

104

In [15]:
def edit(word):   
    return list(set(inserts(word) + delete(word) + swap(word) + replace(word)))

## edit 1

In [21]:
def spell_check_edit_1(word, count = 5):
    
    output = []
    suggested_words = edit(word)
    
    for wrd in suggested_words:        
        if wrd in word_probability.keys():
            output.append([wrd, word_probability[wrd]])
            
    return list(pd.DataFrame(output, columns = ['word','prob']).sort_values(by = 'prob', ascending = False).head(count)['word'].values)

## edit 2

In [19]:
word_probability = {'example': 0.9, 'samples': 0.8, 'examine': 0.7, 'exemplify': 0.6}

def spell_check_edit_2(word, count = 5):
    
    output = []
    suggested_words = edit(word)       # Level one Edit
    
    for e1 in edit(word):
        suggested_words += edit(e1)    # Second Level Edit 
    
    suggested_words = list(set(suggested_words))
    
    for wrd in suggested_words:
        if wrd in word_probability.keys():
            output.append([wrd, word_probability[wrd]])
    return list(pd.DataFrame(output, columns = ['word','prob']).sort_values(by = 'prob', ascending = False).head(count)['word'].values)
        

def check_spelling():
    word = entry.get().strip().lower()  # Get input word
    if not word:
        messagebox.showerror("Error", "Please enter a word.")
        return
    
    suggestions = spell_check_edit_2(word)  # Pass the word argument properly
    
    # Display suggestions in output_label
    suggestions_text = "\n".join(suggestions)
    output_label.config(text=f"Suggestions:\n{suggestions_text}")

## tkinter

In [ ]:
import tkinter as tk
from tkinter import messagebox
root = tk.Tk()
root.title("Spell Checker")
root.geometry("400x300")

# Title Label
title_label = tk.Label(root, text="Spell Checker", font=("Arial", 16, "bold"))
title_label.pack(pady=10)

# Entry Field
entry = tk.Entry(root, font=("Arial", 14), width=25)
entry.pack(pady=10)
# Check Button
check_button = tk.Button(root, text="Check Spelling", command=check_spelling)
check_button.pack(pady=10)
clear_button = tk.Button(root, text="Clear", command=lambda: clear_fields(entry, output_label))
clear_button.pack(pady=5)
output_label = tk.Label(root, text="", font=("Arial", 12), fg="blue")
output_label.pack(pady=10)

# Run Tkinter App
root.mainloop()

## whole code at once

In [ ]:
import tkinter as tk
from tkinter import messagebox
import pandas as pd
from collections import Counter

# Sample word probability dictionary (Replace with real frequency data)
word_probability = Counter({
    "family": 500, "familiar": 300, "fame": 200, "female": 150, "familial": 100
})

# Function to generate edits (deletes, inserts, swaps, replaces)
def edit(word):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    
    deletes = [L + R[1:] for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
    replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
    inserts = [L + c + R for L, R in splits for c in letters]
    
    return set(deletes + transposes + replaces + inserts)

# Spell checker function with probability-based ranking
def spell_check_edit_2(word, count=5):
    output = []
    suggested_words = edit(word)  # Level one edit

    # Second-level edit (for better accuracy)
    for e1 in edit(word):
        suggested_words.update(edit(e1))  

    # Remove duplicates
    suggested_words = list(set(suggested_words))

    # Filter words that exist in word_probability dictionary
    for wrd in suggested_words:
        if wrd in word_probability:
            output.append([wrd, word_probability[wrd]])

    # Sort by probability and return top `count` words
    if output:
        return list(pd.DataFrame(output, columns=['word', 'prob'])
                    .sort_values(by='prob', ascending=False)
                    .head(count)['word'].values)
    return ["No suggestions found"]

# Function to check spelling when button is clicked
def check_spelling():
    word = entry.get().strip().lower()  # Get input word
    if not word:
        messagebox.showerror("Error", "Please enter a word.")
        return
    
    suggestions = spell_check_edit_2(word)  # Pass the word argument properly
    
    # Display suggestions in output_label
    suggestions_text = "\n".join(suggestions)
    output_label.config(text=f"Suggestions:\n{suggestions_text}")

# Tkinter GUI
root = tk.Tk()
root.title("Spell Checker")
root.geometry("400x300")

# Title Label
title_label = tk.Label(root, text="Spell Checker", font=("Arial", 16, "bold"))
title_label.pack(pady=10)

# Entry Field
entry = tk.Entry(root, font=("Arial", 14), width=25)
entry.pack(pady=10)

# Check Button
check_button = tk.Button(root, text="Check Spelling", command=check_spelling)
check_button.pack(pady=10)

# Output Label
output_label = tk.Label(root, text="", font=("Arial", 12), fg="blue")
output_label.pack(pady=10)

# Run Tkinter App
root.mainloop()
